<a href="https://colab.research.google.com/github/abhirockraj/pytorch_nn/blob/main/Finacial_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional as F

In [6]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [7]:
adni = yf.Ticker("ADANIENT.NS")

In [8]:
adni.financials

,2021-03-31,2020-03-31,2019-03-31,2018-03-31
Research Development,None,None,None,None
Effect Of Accounting Charges,None,None,None,None
Income Before Tax,1.38541e+10,1.36432e+10,5.5967e+09,6.587e+09
Minority Interest,1.75144e+10,1.26337e+10,3.8777e+09,7.7786e+09
Net Income,9.2264e+09,1.13817e+10,7.1714e+09,7.5725e+09
Selling General Administrative,6.78426e+10,6.45792e+10,5.08845e+10,3.89138e+10
Gross Profit,9.28931e+10,8.74223e+10,7.05725e+10,5.92558e+10
Ebit,1.98188e+10,1.85189e+10,1.59071e+10,1.38368e+10
Operating Income,1.98188e+10,1.85189e+10,1.59071e+10,1.38368e+10
Other Operating Expenses,-1.397e+08,-3.964e+08,-1.168e+08,-1.34e+08


In [9]:
hist = adni.history(period="5y")

In [10]:
hist.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2016-10-06,36.339389,36.633309,35.350743,35.671387,2998323,0.0,0
2016-10-07,35.885145,36.526428,35.430900,36.259224,5426409,0.0,0
2016-10-10,36.526426,37.595231,35.858424,36.900505,9051818,0.0,0
2016-10-13,36.847070,37.247872,35.831704,36.018745,4018770,0.0,0
2016-10-14,36.232504,36.606587,35.858425,36.125626,3967572,0.0,0


In [227]:
df_x_close_out = hist[['Open','High','Low']]
df_y_close_out = hist['Close']

In [228]:
df_x_close_out.reset_index(drop=True, inplace=True)
df_y_close_out.reset_index(drop=True, inplace=True)

In [229]:
df_x_close_out.shape

(1233, 3)

In [230]:
df_x_close_out_val= df_x_close_out[1:247]
df_y_close_out_val = df_y_close_out[1:247]
df_x_close_out_train= df_x_close_out[247:]
df_y_close_out_train = df_y_close_out[247:]

In [231]:
df_x_close_out_train.shape

(986, 3)

Pytorch model creation

In [232]:
input_size =3
hidden_size=3
lr= 0.001
no_of_epoch=100
batch =1
num_layer=3

In [233]:
df_x_close_out_val =torch.Tensor(df_x_close_out_val.values)
df_y_close_out_val =torch.Tensor(df_y_close_out_val.values)
df_x_close_out_train= torch.Tensor(df_x_close_out_train.values)
df_y_close_out_train= torch.Tensor(df_y_close_out_train.values)

In [234]:
df_x_close_out_train.shape

torch.Size([986, 3])

In [235]:
train_loader= torch.utils.data.TensorDataset(df_x_close_out_train, df_y_close_out_train)
val_loader =torch.utils.data.TensorDataset(df_x_close_out_val,df_y_close_out_val)

In [236]:
train_loader = DataLoader(train_loader,batch_size=batch,shuffle=True)
val_loader = DataLoader(val_loader,batch_size=batch,shuffle=False)

In [237]:
c=0
for x,y in train_loader:
  c+=1
  x = torch.flatten(x,start_dim=1)
  print(x)
  if(c>1):
    break

tensor([[1161.7289, 1190.2100, 1136.9952]])
tensor([[150.0418, 151.6244, 146.7285]])


In [240]:
class NeuralNet(pl.LightningModule):
    def __init__(self,input_size=3,hidden_size=9,num_layer=3):
        super(NeuralNet,self).__init__()
        self.rnn1= nn.RNN(input_size,hidden_size,num_layer,batch_first=True)
        self.linear1=nn.Linear(hidden_size,1)
    def forward(self,x):
        out=self.rnn1(x)
        out=self.linear2(out)
        return out
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=lr)
        return optimizer
    def training_step(self, train_batch, batch_idx):
        x,y = train_batch
        x  = torch.flatten(x,start_dim=1)
        print(x)
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y)
        self.log('train_loss', loss)
        return loss
    def validation_step(self, val_batch, batch_idx):
        x,y = val_batch
        # x = x.view(x.size(0), -1)
        x  = torch.flatten(x,start_dim=1)
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y)
        self.log('val_loss', loss)

In [241]:
trainer = Trainer(fast_dev_run=True)
model = NeuralNet()
trainer.fit(model, train_loader, val_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
Running in fast_dev_run mode: will run a full train, val, test and prediction loop using 1 batch(es).

  | Name    | Type   | Params
-----------------------------------
0 | rnn1    | RNN    | 486   
1 | linear1 | Linear | 10    
-----------------------------------
496       Trainable params
0         Non-trainable params
496       Total params
0.002     Total estimated model params size (MB)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:327: UserWarning: The number of training samples (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  f"The number of training samples ({self.num_training_batches}) is smaller than the logging interval"


Training: -1it [00:00, ?it/s]

tensor([[90.7159, 97.5481, 89.2293]])


RuntimeError: ignored

In [69]:
a= np.array([[1,2,3],[3,2,1]])
a.shape

(2, 3)

In [73]:
a.reshape(-1,3)

array([[1, 2, 3],
       [3, 2, 1]])

In [74]:
a.shape

(2, 3)

In [85]:
b= (2,1)

In [86]:
x,y= b

In [87]:
print(x)

2
